In [1]:
from hashlib import md5
import os
import pickle

# 对数据进行MD5哈希

In [2]:
sql = '''
    SELECT * 
    FROM table_a
    WHERE ds='20230819' and user_id != ""
    GROUP BY SPLIT_PART(user_id, '_', 1)
'''

In [3]:
hash_key = md5(sql.encode('utf-8')).hexdigest()
hash_key

'1536ce5208a9577d67783b8f7423b516'

# 用hash_key作为cache的键 对sql的结果进行cache
在执行sql很复杂代价很大时，将执行的结果cache是一个很好的策略。而对于sql串的string对象采用哈希作为唯一key是进行储存的合适策略。  
若结果很大应该存在hdfs等平台，不该使用pickle方法；若结果较小则使用pickle方法很合适。

In [ ]:
class MyResults(object):
    def __init__(self, cache_file_path='./.cache_File.pkl'):
        self.cache_file_path = cache_file_path
        self.cache = self._load_cache()
        # 在初始化函数中进行load_cache过程

    def _load_cache(self):
        """
        按照self.cache_file_path路径加载cache的pickle文件
        加载的文件是一个dict 以md5哈希值作为key sql结果作为value值
        :return:
        """
        if os.path.exists(self.cache_file_path):
            with open(self.cache_file_path, 'rb') as f:
                cache = pickle.load(f)
            print('load cache: {}'.format(len(cache)))
        else:
            cache = {}
        return cache

    def _save_cache(self):
        """
        将self.cache对象用pickle存储本地文件
        存储的对象是一个dict 以md5哈希值为key sql结果为value值
        :return:
        """
        with open(self.cache_file_path, 'wb') as f:
            pickle.dump(self.cache, f)

    def _update_cache(self, hash_key, res):
        """
        按照哈希值更新对应的结果
        :param hash_key: 哈希值
        :param res: sql结果
        :return: 更新后的cache对象
        """
        cache = self._load_cache()
        cache[hash_key] = res
        with open(self.cache_file_path, 'wb') as f:
            pickle.dump(cache, f)
        return cache

    def do_sql(self, sql_string):
        """
        对sql_string进行执行 如果存在在cache中就返回 否则调用api去执行
        :param sql_string: sql执行语句
        :return:
        """
        hash_key = md5(sql_string.encode('utf-8')).hexdigest()
        if hash_key in self.cache.keys():
            res = self.cache.get(hash_key)
        else:
            # 调用对应的sql_api
            res = ""
        return res

    def function(self, sql_string):
        """
        按照sql_string执行方法 并计算hash_key 存储在cache中，最后写入pickle文件中
        :param sql_string: sql串对应的string对象
        :return:
        """
        result = self.do_sql(sql_string)
        hash_key = md5(sql_string.encode('utf-8')).hexdigest()
        self.cache[hash_key] = result
        self._save_cache()
